In [123]:
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import json
from scipy import stats
import pandas as pd
import hvplot.pandas
import gmaps
from config import geoapify_key, open_weather_key 

In [50]:
df = pd.read_csv("HMEOpenLargeJobs_20231202.csv")
df.head(2)

,ProjectID,Project-Number,Name,Address,Geometry,LocationJSON
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{""location"":{""lat"":39.805279,""lng"":-104.958759...","{""Street1"":""5950 York St"",""Street2"":null,""City..."
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{""location"":{""lat"":38.736055,""lng"":-104.737949...","{""Street1"":""6510 Southmoor Dr"",""Street2"":null,..."


In [51]:
geometry_json = df['Geometry'].apply(json.loads).apply(pd.Series)
df = pd.concat([df, geometry_json], axis=1).drop('Geometry', axis=1)

# Normalize the 'LocationJSON' column from JSON string to individual columns

location_json = df['LocationJSON'].apply(json.loads).apply(pd.Series)
df = pd.concat([df, location_json], axis=1).drop('LocationJSON', axis=1)

# Display the normalized DataFrame
df = pd.DataFrame(df)
df.head(1)

,ProjectID,Project-Number,Name,Address,location,viewport,Street1,Street2,City,State,Zip,Country,GooglePlaceID,CityStateString,Address_HTML,Address_Formatted,Address_Vicinity,DistanceToHME,Geometry
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{'lat': 39.805279, 'lng': -104.958759}","{'east': -104.9576728197085, 'north': 39.80663...",5950 York St,None,Denver,CO,80216,USA,ChIJPYAhr-t5bIcR9x-Dourt6BM,"Denver, CO","<span class=""street-address"">5950 York St</spa...","5950 York St, Denver, CO 80216, USA",Denver,"{""DistanceMeters"":865917,""DistanceMiles"":538.0...","{""location"":{""lat"":39.805279,""lng"":-104.958759..."


In [52]:
df_normalized = pd.json_normalize(df['location'])

# Concatenate the normalized columns with the original DataFrame
df_result = pd.concat([df, df_normalized], axis=1)

# Drop the original 'location' column
df_result = df_result.drop('location', axis=1)

# Display the resulting DataFrame
df = pd.DataFrame(df_result)
df.head(3)

,ProjectID,Project-Number,Name,Address,viewport,Street1,Street2,City,State,Zip,Country,GooglePlaceID,CityStateString,Address_HTML,Address_Formatted,Address_Vicinity,DistanceToHME,Geometry,lat,lng
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{'east': -104.9576728197085, 'north': 39.80663...",5950 York St,None,Denver,CO,80216,USA,ChIJPYAhr-t5bIcR9x-Dourt6BM,"Denver, CO","<span class=""street-address"">5950 York St</spa...","5950 York St, Denver, CO 80216, USA",Denver,"{""DistanceMeters"":865917,""DistanceMiles"":538.0...","{""location"":{""lat"":39.805279,""lng"":-104.958759...",39.805279,-104.958759
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{'east': -104.73519499999998, 'north': 38.7379...",6510 Southmoor Dr,None,Fountain,CO,80817-1045,USA,ChIJD2Ah-ZJBE4cRqyATJnTcj0Y,"Fountain, CO","<span class=""street-address"">6510 Southmoor Dr...","6510 Southmoor Dr, Fountain, CO 80817, USA",Fountain,"{""DistanceMeters"":847489,""DistanceMiles"":526.6...","{""location"":{""lat"":38.736055,""lng"":-104.737949...",38.736055,-104.737949
2,87538,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA","{'east': -97.34728431970848, 'north': 32.89572...",8776 Blue Mound Rd,None,Fort Worth,TX,76131-3304,USA,Ei04Nzc2IEJsdWUgTW91bmQgUmQsIEZvcnQgV29ydGgsIF...,"Fort Worth, TX","<span class=""street-address"">8776 Blue Mound R...","8776 Blue Mound Rd, Fort Worth, TX 76131, USA",Fort Worth,"{""DistanceMeters"":788571,""DistanceMiles"":489.9...","{""location"":{""lat"":32.8943727,""lng"":-97.348633...",32.894373,-97.348633


##### Drop NAN values in Latitude and Longitudes

In [53]:
df = df.dropna(subset=['lat', 'lng'])
df.head(3)

,ProjectID,Project-Number,Name,Address,viewport,Street1,Street2,City,State,Zip,Country,GooglePlaceID,CityStateString,Address_HTML,Address_Formatted,Address_Vicinity,DistanceToHME,Geometry,lat,lng
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{'east': -104.9576728197085, 'north': 39.80663...",5950 York St,None,Denver,CO,80216,USA,ChIJPYAhr-t5bIcR9x-Dourt6BM,"Denver, CO","<span class=""street-address"">5950 York St</spa...","5950 York St, Denver, CO 80216, USA",Denver,"{""DistanceMeters"":865917,""DistanceMiles"":538.0...","{""location"":{""lat"":39.805279,""lng"":-104.958759...",39.805279,-104.958759
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{'east': -104.73519499999998, 'north': 38.7379...",6510 Southmoor Dr,None,Fountain,CO,80817-1045,USA,ChIJD2Ah-ZJBE4cRqyATJnTcj0Y,"Fountain, CO","<span class=""street-address"">6510 Southmoor Dr...","6510 Southmoor Dr, Fountain, CO 80817, USA",Fountain,"{""DistanceMeters"":847489,""DistanceMiles"":526.6...","{""location"":{""lat"":38.736055,""lng"":-104.737949...",38.736055,-104.737949
2,87538,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA","{'east': -97.34728431970848, 'north': 32.89572...",8776 Blue Mound Rd,None,Fort Worth,TX,76131-3304,USA,Ei04Nzc2IEJsdWUgTW91bmQgUmQsIEZvcnQgV29ydGgsIF...,"Fort Worth, TX","<span class=""street-address"">8776 Blue Mound R...","8776 Blue Mound Rd, Fort Worth, TX 76131, USA",Fort Worth,"{""DistanceMeters"":788571,""DistanceMiles"":489.9...","{""location"":{""lat"":32.8943727,""lng"":-97.348633...",32.894373,-97.348633


In [16]:
endpoint = 'https://api.geoapify.com/v1/geocode/search'
params = {
    'text': df['Address'].iloc[0],
    'apiKey': geoapify_key,
}
response = requests.get(endpoint, params=params)

# Print the response 
print(response.json())

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'country': 'United States', 'country_code': 'us', 'state': 'Colorado', 'county': 'Adams County', 'postcode': '80216', 'street': 'York Street', 'housenumber': '5950', 'lon': -104.95957296999853, 'lat': 39.80748322631706, 'state_code': 'CO', 'formatted': '5950 York Street, Adams County, CO 80216, United States of America', 'address_line1': '5950 York Street', 'address_line2': 'Adams County, CO 80216, United States of America', 'timezone': {'name': 'America/Denver', 'offset_STD': '-07:00', 'offset_STD_seconds': -25200, 'offset_DST': '-06:00', 'offset_DST_seconds': -21600, 'abbreviation_STD': 'MST', 'abbreviation_DST': 'MDT'}, 'plus_code': '85FQR24R+X5', 'result_type': 'building', 'rank': {'importance': -0.8399999999999999, 'popularity'

In [15]:
import gmaps

# Set up the gmaps API key
gmaps.configure(api_key=geoapify_key)

# Create a map centered at the mean of your data
fig = gmaps.figure(center=(df['lat'].mean(), df['lng'].mean()), zoom_level=10)
locations = df[['lat', 'lng']] 
locations = locations[locations['lng']<90] 
# Create a heatmap layer 
fig.add_layer(gmaps.heatmap_layer(locations))

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

ERROR: Could not find a version that satisfies the requirement jupyter-gmap (from versions: none)
ERROR: No matching distribution found for jupyter-gmap


##### Get Weather By Address

In [34]:
import requests

def get_weather(api_key, city, country_code='', units='metric'):
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    
    # Construct the API request URL
    params = {
        'q': f'{city},{country_code}',
        # 'q': df['Address'].iloc[0],
        'units': units,
        'appid': api_key,
        
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Parse the JSON response
        weather_data = response.json()

        # Extract relevant information
        temperature = weather_data['main']['temp']
        description = weather_data['weather'][0]['description']
        humidity = weather_data['main']['humidity']

        # Print or return the weather information
        print(f'Temperature: {temperature}°C')
        print(f'Description: {description}')
        print(f'Humidity: {humidity}%')
    else:
        print(f'Error {response.status_code}: {response.text}') 

# Replace 'CITY_NAME' with the city you want to get weather information for
print(df['Address'].iloc[0])
print(df['Address'].iloc[0].split(',')[0].strip())
print(df['Address'].iloc[0].split(',')[1].strip())
print(df['Address'].iloc[0].split(',')[3].strip())

city_name = df['Address'].iloc[0].split(',')[1].strip()

# Replace 'COUNTRY_CODE' with the country code if necessary (e.g., 'US', 'IN')
country_code = df['Address'].iloc[0].split(',')[3].strip()

# Call the function to get weather information
get_weather(open_weather_key, city_name, country_code) 

5950 York St, Denver, CO 80216, USA
5950 York St
Denver
USA
Temperature: -0.65°C
Description: overcast clouds
Humidity: 69%


##### Get Weather by Coordinates

In [40]:
def get_weather(api_key, latitude, longitude):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'lat': latitude,
        'lon': longitude,
        'appid': api_key,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        weather_data = response.json()
        return weather_data
    else:
        print(f"Error: {response.status_code}")
        return None

# Replace 'YOUR_API_KEY' with the actual API key you obtained from OpenWeatherMap
api_key = 'YOUR_API_KEY'
latitude = df['lat'].iloc[0]  # First occurance
longitude = df['lng'].iloc[0]  # First Occurance

weather_info = get_weather(open_weather_key, latitude, longitude)
print(weather_info)
print()
if weather_info:
    print("Weather Information:")
    print(f"Temperature: {weather_info['main']['temp']}°C")
    print(f"Description: {weather_info['weather'][0]['description']}")

{'coord': {'lon': -104.9588, 'lat': 39.8053}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': -0.59, 'feels_like': -2.49, 'temp_min': -2.59, 'temp_max': 3.94, 'pressure': 1021, 'humidity': 69}, 'visibility': 10000, 'wind': {'speed': 1.54, 'deg': 260}, 'clouds': {'all': 0}, 'dt': 1704089431, 'sys': {'type': 2, 'id': 2034152, 'country': 'US', 'sunrise': 1704032453, 'sunset': 1704066260}, 'timezone': -25200, 'id': 5417737, 'name': 'Commerce City', 'cod': 200}

Weather Information:
Temperature: -0.59°C
Description: clear sky


In [69]:
import requests
import pandas as pd

def get_weather(api_key, latitude, longitude):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "lat": latitude,
        "lon": longitude,
        "appid": api_key,
        "units": "metric",  # You can change this to imperial for Fahrenheit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        weather_data = response.json()
        return weather_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

# Replace 'YOUR_API_KEY' with the actual API key you obtained from OpenWeatherMap
api_key = open_weather_key 

# Get weather data for each row in the DataFrame
weather_list = []
for index, row in df.iterrows():
    latitude = row['lat']
    longitude = row['lng']
    ProjectID = row['ProjectID']
    weather_data = get_weather(api_key, latitude, longitude)

    if weather_data:
        weather_list.append({
            'ProjectID': ProjectID,
            'Latitude': latitude,
            'Longitude': longitude,
            'Temperature': weather_data['main']['temp'],
            'Pressure': weather_data['main']['pressure'],
            'Humidity': weather_data['main']['humidity'],
            'Description': weather_data['weather'][0]['description'],
            'DateTime': weather_data['dt'],
            'WindSpeed': weather_data['wind']['speed'],
            'Clouds': weather_data['clouds']['all'],
        })

# Create a new DataFrame with weather information
weather_df = pd.DataFrame(weather_list)

# Print the resulting DataFrame
weather_df.head()


,ProjectID,Latitude,Longitude,Temperature,Pressure,Humidity,Description,DateTime,WindSpeed,Clouds
0,85216,39.805279,-104.958759,-0.66,1020,68,clear sky,1704095017,0.00,0
1,87261,38.736055,-104.737949,-2.92,1021,79,clear sky,1704095018,2.06,0
2,87538,32.894373,-97.348633,2.93,1023,76,clear sky,1704095019,7.72,0
3,87544,33.383391,-96.282564,0.96,1023,84,scattered clouds,1704095019,5.36,48
4,88609,41.240430,-96.011843,-6.97,1028,88,clear sky,1704095020,1.54,0


In [68]:
weather_df['Description'].unique()

array(['clear sky', 'broken clouds', 'few clouds', 'mist',
       'overcast clouds', 'fog', 'scattered clouds'], dtype=object)

##### Merge Datasets

In [73]:
df_merged = pd.merge(df, weather_df, how='left', on='ProjectID')
df_merged.head(3)

,ProjectID,Project-Number,Name,Address,viewport,Street1,Street2,City,State,Zip,...,lng,Latitude,Longitude,Temperature,Pressure,Humidity,Description,DateTime,WindSpeed,Clouds
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{'east': -104.9576728197085, 'north': 39.80663...",5950 York St,None,Denver,CO,80216,...,-104.958759,39.805279,-104.958759,-0.66,1020,68,clear sky,1704095017,0.00,0
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{'east': -104.73519499999998, 'north': 38.7379...",6510 Southmoor Dr,None,Fountain,CO,80817-1045,...,-104.737949,38.736055,-104.737949,-2.92,1021,79,clear sky,1704095018,2.06,0
2,87538,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA","{'east': -97.34728431970848, 'north': 32.89572...",8776 Blue Mound Rd,None,Fort Worth,TX,76131-3304,...,-97.348633,32.894373,-97.348633,2.93,1023,76,clear sky,1704095019,7.72,0


#### Drop duplicate columns eg latitudes and longitudes

##### UNIX timestamp to show the exact time current weather was being extracted

In [83]:
import datetime
df_merged['TimeStamp'] = df_merged['DateTime'].apply(lambda x: datetime.datetime.fromtimestamp(x))
df_merged.head(3)

,ProjectID,Project-Number,Name,Address,viewport,Street1,Street2,City,State,Zip,...,Latitude,Longitude,Temperature,Pressure,Humidity,Description,DateTime,WindSpeed,Clouds,TimeStamp
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{'east': -104.9576728197085, 'north': 39.80663...",5950 York St,None,Denver,CO,80216,...,39.805279,-104.958759,-0.66,1020,68,clear sky,1704095017,0.00,0,2024-01-01 10:43:37
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{'east': -104.73519499999998, 'north': 38.7379...",6510 Southmoor Dr,None,Fountain,CO,80817-1045,...,38.736055,-104.737949,-2.92,1021,79,clear sky,1704095018,2.06,0,2024-01-01 10:43:38
2,87538,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA","{'east': -97.34728431970848, 'north': 32.89572...",8776 Blue Mound Rd,None,Fort Worth,TX,76131-3304,...,32.894373,-97.348633,2.93,1023,76,clear sky,1704095019,7.72,0,2024-01-01 10:43:39


In [120]:
df_merged["LabelColor"] = ["yellow" if x > 25 else "green" for x in df_merged["WindSpeed"]]

In [121]:
df_merged["LabelColor"].unique()

array(['green'], dtype=object)

In [89]:
df_merged["colorlabel"] = ["yellow" if x > 4 else "blue" for x in df_merged["WindSpeed"]]

##### Show on map

##### The number of projects with wind against those without wind

In [117]:
with_wind = (df_merged["WindSpeed"]>0).sum()
without_wind = (df_merged["WindSpeed"]==0).sum()
print("The projects with wind are {} while without wind are {}".format(with_wind,without_wind))

The projects with wind are 212 while without wind are 22


In [ ]:
# Create a bar chart using Seaborn
sns.barplot(x=categories, y=values)

# Add labels and title
plt.xlabel('Categories')
plt.ylabel('Values')
plt.title('Bar Chart Example')

# Show the plot
plt.show()

In [126]:
(df_merged["WindSpeed"]>25).sum()
#There are no projects with wind speed more than 25mph hence bar graph not relevant.
#No project is delayed as windspeed not more than 25mph
#Line graph not possible from data

0

###### 